# 23.04.26(수) 20, 30, 40 직선구간에서만 데이터 뽑기 [good] 

## 23.04.25 rsr 파일을 _rsr.csv로 만들고, 각각의 도로별로 시간 average time 구하는 코드
경로만 바꾸면 됨\
VehType또한 기준으로 포함시킴. 그러나 925와 926은 HGV로 구분

## 1. 아래 코드는 왜 그런지 모르겠는데, 마지막 폴더에 몰빵으로 다 저장됨

In [143]:
import os
import csv
from collections import defaultdict


# 입력받은 경로를 지정합니다.
base_folder = r'D:\Ramp Data'

# 폴더 생성을 위한 구간별 값들을 지정합니다.
ranges = ['10', '20', '30']
combinations = [f'{x},{y}' for x in ranges for y in ranges]


# 모든 폴더에 대해 반복
for combination in combinations:
    folder_path = os.path.join(base_folder, combination)
    # XXX에 해당하는 숫자를 리스트로 만듭니다.
    ##### xxx_list = [str(i).zfill(3) for i in range(58, 234)] # 직선구간 
    if combination == '10,10':
        xxx_list = [str(i).zfill(3) for i in range(234, 410)] # 10,10
    elif combination == '10,20':
        xxx_list = [str(i).zfill(3) for i in range(410, 586)] # 10,20
    elif combination == '10,30':
        xxx_list = [str(i).zfill(3) for i in range(586, 762)] # 10,30
    elif combination == '20,10':
        xxx_list = [str(i).zfill(3) for i in range(762, 938)] # 20,10
    elif combination == '20,20':
        xxx_list = [str(i).zfill(3) for i in range(938, 1114)] # 20,20 #############
    elif combination == '20,30':
        xxx_list = [str(i).zfill(3) for i in range(1114, 1290)] # 20,30
    elif combination == '30,10':
        xxx_list = [str(i).zfill(3) for i in range(1290, 1466)] # 30,10
    elif combination == '30,20':
        xxx_list = [str(i).zfill(3) for i in range(1466, 1642)] # 30,20
    elif combination == '30,30':
        xxx_list = [str(i).zfill(3) for i in range(1642, 1818)] # 30,30##############################################



    # 각 XXX에 대해 A 값을 구합니다.
    a_dict = {}
    b_dict = {}
    for i, xxx in enumerate(xxx_list):
        a_dict[xxx] = str(i//11 + 20)
        b_dict[xxx] = str((i%11)*  10)


    # 각 Y에 대해 C 값을 구합니다.
    c_dict = {'1': '1', '2': '2', '3': '3', '4':'4', '5':'5'}
    #c_dict = {'1': '1', '2': '2', '3': '3', '4':'4', '5':'5', '6':'6', '7':'7', '8':'8', '9':'9'}

    # rsr_files 폴더 내 모든 파일 경로 가져오기
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.rsr'):
                file_paths.append(os.path.join(root, file))


    # 각 파일별로 csv 파일 생성
    for file_path in file_paths:
        # XXX 값을 추출합니다.
        xxx = os.path.basename(file_path)[:7]

        # A, B, C 값을 추출합니다. 
        number_check = xxx[3:7]
        if number_check[0] == '0':
            a_value = a_dict[xxx[4:7]] # 20~35  3자릿수 
            b_value = b_dict[xxx[4:7]] # 0~100  3자릿수  
        else :
            a_value = a_dict[xxx[3:7]] # 20~35   4자릿수 
            b_value = b_dict[xxx[3:7]] # 0~100   4자릿수  
    #     print("a_value, b_value ; ", a_value, '       ', b_value)

        c_value = c_dict[os.path.basename(file_path)[10]]
        if(int(c_value)>5):
            print('c_value : ', c_value)

        # 새 파일 이름을 만듭니다.
        csv_name = f"{a_value};{b_value};{c_value}"
        csv_path = os.path.join(root, csv_name + '.csv')


        # 파일 열기
        with open(file_path, 'r', encoding='utf-8') as f:
            # 첫 8줄은 건너뛰기
            for i in range(9):
                next(f)
            # CSV 파일로 저장
            with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=',')
                # 헤더 정보 추가
                csvwriter.writerow(['Time', 'No', 'Veh', 'VehType', 'TravTm', 'Delay', 'Dist'])
                # 데이터 추가
                data = []
                time_by_no = defaultdict(list)
                for line in f:
                    row = line.strip().split(';')
                    data.append(row)
                    time = float(row[4])
                    no = int(row[1])
                    veh_type = int(row[3])
                    time_by_no[no, veh_type].append(time)
                    csvwriter.writerow(row)


        # 별도의 csv 파일 생성
        avg_time_path = os.path.join(root, csv_name +'_avg_time.csv')
        with open(avg_time_path, 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=',')
            # 헤더 정보 추가
            csvwriter.writerow(['No', 'VehType', csv_name])
            # 각 기준에 따른 TravTm 평균 구하기
    #         no_criteria = [20, 30, 40] ##############################################################

            no_criteria = [21,22,23,24,31,32,33,34,41,42,43,44]#############################################################
            veh_type_criteria = [100, 925, 926, 'HGV','AVG']

            # 각 기준에 따른 TravTm 평균 구하기
            for no in no_criteria:
                for veh_type in veh_type_criteria:
                    times_925 = time_by_no[no, 925]
                    times_926 = time_by_no[no, 926]
                    times_100 = time_by_no[no, 100]

                    if veh_type == 'HGV':
                        if len(times_925) > 0 or len(times_926) > 0:
                            avg_time = (sum(times_925) + sum(times_926)) / (len(times_925) + len(times_926))
                        else:
                            avg_time = 0
                        csvwriter.writerow([no, veh_type, avg_time])

                    elif veh_type==100:
                        if len(times_100) > 0:
                            avg_time = sum(times_100) / len(times_100)
                        else:
                            avg_time = 0
                        csvwriter.writerow([no, 'LOV', avg_time])

                    elif veh_type=='AVG':
                        if len(times_925) > 0 or len(times_926) > 0 or len(times_100):
                            avg_time = (sum(times_925) + sum(times_926) + sum(times_100)) / (len(times_925) + len(times_926) + len(times_100))
                        else:
                            avg_time = 0
                        csvwriter.writerow([no, 'AVG', avg_time])
    print(combination, '완료')
print('모든파일 완료')

20,30 완료
30,10 완료
30,20 완료
30,30 완료
모든파일 완료


## (X) 하위 CSV파일 모두 없애는 코드

In [48]:
import os
import glob

# 제거할 파일 확장자를 지정합니다.
extension = '.csv'

# 제거할 파일이 있는 폴더를 지정합니다.
folder_path = r'D:\0522Results(20,30,40 with CC2,CC4)'

# 하위폴더까지 포함하여 모든 csv 파일의 경로를 찾습니다.
csv_files = glob.glob(os.path.join(folder_path, '**', f'*{extension}'), recursive=True)

# 모든 csv 파일을 제거합니다.
for file_path in csv_files:
    os.remove(file_path)

print('제거 완료')

제거 완료


## 2. (O) 모든 _avg_time.csv 파일 합치기

In [145]:
import os
import pandas as pd

# 입력받은 경로를 지정합니다.
base_folder = r'D:\Ramp Data'

# 폴더 생성을 위한 구간별 값들을 지정합니다.
ranges = ['10', '20', '30']
combinations = [f'{x},{y}' for x in ranges for y in ranges]


# 모든 폴더에 대해 반복
for combination in combinations:
    directory = os.path.join(base_folder, combination)
    print("directory : ",directory)  # D:\Ramp Data\20,30 일걸?
    
    # 모든 _avg_time.csv 파일을 찾기 위한 경로 설정
    search_term = '_avg_time.csv'
    all_avg_time_path = os.path.join(directory, 'all_avg_time.csv')

    # all_avg_time.csv 파일이 존재하면 삭제
    if os.path.exists(all_avg_time_path):
        os.remove(all_avg_time_path)

    # 모든 _avg_time.csv 파일을 읽어서 데이터프레임으로 변환하여 리스트에 저장
    df_list = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(search_term):
                filepath = os.path.join(root, filename)
                temp_df = pd.read_csv(filepath, index_col=[0,1])
                df_list.append(temp_df)

    # 리스트에 있는 모든 데이터프레임을 병합하여 all_avg_time.csv 파일로 저장
    all_df = pd.concat(df_list, axis=1)
    all_df.to_csv(all_avg_time_path)
    print(combination, '완료')
    
print('전체 완료')


directory :  D:\Ramp Data\10,10
10,10 완료
directory :  D:\Ramp Data\10,20
10,20 완료
directory :  D:\Ramp Data\10,30
10,30 완료
directory :  D:\Ramp Data\20,10
20,10 완료
directory :  D:\Ramp Data\20,20
20,20 완료
directory :  D:\Ramp Data\20,30
20,30 완료
directory :  D:\Ramp Data\30,10
30,10 완료
directory :  D:\Ramp Data\30,20
30,20 완료
directory :  D:\Ramp Data\30,30
30,30 완료
완료


### (X) Truck Mixed %를 몇%인지 입력하면 그 %에 해당하는 열만 뽑아서 새로운 Csv로 저장해주는 코드

In [169]:
import pandas as pd

# 데이터 파일 경로
data_path = r"C:\Users\user\Desktop\23.04.15\0425Results\all_avg_time.csv"

# B값 입력 입력하는곳!!!!!!!!!!!!!!!
B = '10'

# 데이터 불러오기
df = pd.read_csv(data_path)

# B값에 해당하는 column들만 추출하여 새로운 DataFrame 생성
filtered_df = df.loc[:, ["No", "VehType"] + [col for col in df.columns[2:] if B in col]]

# 결과를 새로운 csv 파일로 저장
filtered_df.to_csv(rf"C:\Users\user\Desktop\23.04.15\0425Results\{B}_avg_time.csv", index=False)


### (O) 3. 0~100 까지 Truck 비율을10단위로 증가시키면서 각각 파일 저장하는 코드

In [153]:
# 입력받은 경로를 지정합니다.
base_folder = r'D:\Ramp Data'

# 폴더 생성을 위한 구간별 값들을 지정합니다.
ranges = ['10', '20', '30']
combinations = [f'{x},{y}' for x in ranges for y in ranges]


# 모든 폴더에 대해 반복
for combination in combinations:
    directory = os.path.join(base_folder, combination)  # D:\Ramp Data\20,30 일걸?
    data_path = os.path.join(directory, 'all_avg_time.csv') # r"D:\Ramp Data\20,30\all_avg_time.csv" 여야 하는데
    print(data_path) #############################################################################


    # B값 0~100 까지 10씩 증가시키면서
    for B in range(0, 101, 10):

        B = str(B)
        # 데이터 불러오기
        df = pd.read_csv(data_path)

        # B값에 해당하는 column들만 추출하여 새로운 DataFrame 생성
        filtered_df = pd.DataFrame({"index": []})
        filtered_df["index"] = df[["No", "VehType"]].apply(lambda x: '_'.join(map(str, [x["No"], B + "%", x["VehType"]])), axis=1)

        for col in df.columns[2:]:
            a, b, c = col.split(";")
            if b == B:
                filtered_df[col] = df[col]

        # 결과를 새로운 csv 파일로 저장
        new_name = ';' + B + ';_avg_time.csv'
        new_data_path = os.path.join(directory, new_name) ###########################
        print("new_data_path : ", new_data_path)
        filtered_df.to_csv(new_data_path, index=False)


    from datetime import datetime
    now = datetime.now()
    print("현재 : ", now, ' 에 ', combination,  ' 완료')


D:\Ramp Data\10,10\all_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;0;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;10;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;20;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;30;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;40;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;50;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;60;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;70;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;80;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;90;_avg_time.csv
new_data_path :  D:\Ramp Data\10,10\;100;_avg_time.csv
현재 :  2023-06-04 02:41:07.918174  에  10,10  완료
D:\Ramp Data\10,20\all_avg_time.csv
new_data_path :  D:\Ramp Data\10,20\;0;_avg_time.csv
new_data_path :  D:\Ramp Data\10,20\;10;_avg_time.csv
new_data_path :  D:\Ramp Data\10,20\;20;_avg_time.csv
new_data_path :  D:\Ramp Data\10,20\;30;_avg_time.csv
new_data_path :  D:\Ramp Data\10,20\;40;_avg_time.csv
new_data_path :  D

# 23.05.16(화) 세로로 Volume과 time 저장하기

In [157]:
import os
import pandas as pd

# 입력받은 경로를 지정합니다.
base_folder = r'D:\Ramp Data'

# 폴더 생성을 위한 구간별 값들을 지정합니다.
ranges = ['10', '20', '30']
combinations = [f'{x},{y}' for x in ranges for y in ranges]
dash_combinations = [f'{x}-{y}' for x in ranges for y in ranges]

# 모든 폴더에 대해 반복
for combination, dash_combination in zip(combinations, dash_combinations):
    directory = os.path.join(base_folder, combination)
    result_folder = os.path.join(directory, 'result')

    # result 폴더가 없는 경우 생성
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    data_path = os.path.join(directory, 'all_avg_time.csv')

    # B값 0~100 까지 10씩 증가시키면서
    for B in range(0, 101, 10):
        B = str(B)

        # 데이터 불러오기
        df = pd.read_csv(data_path)

        # B값에 해당하는 column들만 추출하여 새로운 DataFrame 생성
        filtered_df = pd.DataFrame({"index": []})
        filtered_df["index"] = df[["No", "VehType"]].apply(lambda x: '_'.join(map(str, [x["No"], B + "%", x["VehType"]])), axis=1)

        # 원하는 truck %만 filtered_df에 포함시킴
        for col in df.columns[2:]:
            a, b, c = col.split(";")
            if b == B:
                filtered_df[col] = df[col]

        # 결과를 새로운 csv 파일로 저장
        for index, row in filtered_df.iterrows():
            parts = row["index"].split("_")
            X = (int)(parts[0])  # 20_100%_HOV 에서 20
            Z = parts[2]
            if Z == 'AVG':
                continue

            new_df = filtered_df.loc[[index]].copy() # dataframe 복사

            for col in filtered_df.columns[1:]:
                a, b, c = col.split(";")

                # 첫번째 열 값 변경
                if X // 10 == 2:
                    new_col_name = int(a) % 19 * 400
                elif X // 10 == 3:
                    new_col_name = int(a) % 19 * 550
                elif X // 10 == 4:
                    new_col_name = int(a) % 19 * 700

                new_df.rename(columns={col: new_col_name}, inplace=True)

            new_name = str(X) + '_' + B + '_' + dash_combination + '_' + Z + '.csv'
            file_name = os.path.join(result_folder, new_name)

            transposed_df = new_df.transpose()
            transposed_df.to_csv(file_name, index=True, header=False)

    from datetime import datetime
    now = datetime.now()
    print("현재 : ", now, ' 에', combination ,'완료')
    
print('모두 완료')

현재 :  2023-06-04 02:59:17.720860  에 완료
현재 :  2023-06-04 02:59:21.681226  에 완료
현재 :  2023-06-04 02:59:25.550820  에 완료
현재 :  2023-06-04 02:59:29.680759  에 완료
현재 :  2023-06-04 02:59:33.437767  에 완료
현재 :  2023-06-04 02:59:37.265612  에 완료
현재 :  2023-06-04 02:59:41.343703  에 완료
현재 :  2023-06-04 02:59:45.294798  에 완료
현재 :  2023-06-04 02:59:50.249169  에 완료


# 파일 지우기 세트

## 파일 복사 (하위 모든 폴더 탐색하여) : 에러뜨는거는 이미 옮겨놓고 그 안에 폴더에 또 들어감..;;

In [16]:
import os
import shutil

original_path = rf"D:\Ramp Data\HGV\32"
destination_path = rf"D:\Ramp Data\HGV\32\10-10"  # 목적지 경로를 지정하세요

# original_path에서 모든 하위 디렉토리 탐색
for root, dirs, files in os.walk(original_path):
    for file in files:
        if file.endswith(".csv"):
            # 파일 이름 분리하기
            file_name = os.path.splitext(file)[0]
            components = file_name.split("_")
            
            # A, B, C, D, XXX 추출
            A = components[0]
            B = components[1]
            C = components[2].split("-")[0]
            D = components[2].split("-")[1]
            XXX = components[3]
            
            
            # A가 21인 경우 해당 파일을 목적지로 복사
            if A == '32' and XXX == 'HGV' and C == '10' and D == '10':
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(destination_path, file)
                shutil.copyfile(source_file_path, destination_file_path)


SameFileError: 'D:\\Ramp Data\\HGV\\32\\10-10\\32_0_10-10_HGV.csv' and 'D:\\Ramp Data\\HGV\\32\\10-10\\32_0_10-10_HGV.csv' are the same file

## 파일 복사 (original_path에 있는 csv 파일들만)

In [71]:
import os
import shutil

original_path = rf"D:\Ramp Data\32"
destination_path = rf"D:\Ramp Data\32\10-30"  # 목적지 경로를 지정하세요

# original_path에서 파일 검색
for file in os.listdir(original_path):
    if file.endswith(".csv"):
        # 파일 이름 분리하기
        file_name = os.path.splitext(file)[0]
        components = file_name.split("_")
        
        # A, B, C, D, XXX 추출
        A = components[0]
        B = components[1]
        C = components[2].split("-")[0]
        D = components[2].split("-")[1]
        XXX = components[3]
        
        # A가 21인 경우 해당 파일을 목적지로 복사
        if C == '10' and D == '30':
            source_file_path = os.path.join(original_path, file)
            destination_file_path = os.path.join(destination_path, file)
            shutil.copyfile(source_file_path, destination_file_path)
print('완료')

## csv 파일 다 지워버리기

In [138]:
import os
import glob

# 제거할 파일 확장자를 지정합니다.
extension = '.csv'

# 입력받은 경로를 지정합니다.
base_folder = r'D:\Ramp Data'

# 폴더 생성을 위한 구간별 값들을 지정합니다.
ranges = ['10', '20', '30']
combinations = [f'{x},{y}' for x in ranges for y in ranges]

# 모든 폴더에 대해 반복하여 파일 제거 작업 수행
for combination in combinations:
    folder_path = os.path.join(base_folder, combination)
    os.makedirs(folder_path, exist_ok=True)  # 폴더 생성, 이미 존재하는 경우 무시합니다.

    # 하위 폴더까지 포함하여 모든 csv 파일의 경로를 찾습니다.
    csv_files = glob.glob(os.path.join(folder_path, '**', f'*{extension}'), recursive=True)

    # 모든 csv 파일을 제거합니다.
    for file_path in csv_files:
        os.remove(file_path)

    print(f'{folder_path} 폴더의 제거 완료')

print('모든 작업 완료')

D:\Ramp Data\10,10 폴더의 제거 완료
D:\Ramp Data\10,20 폴더의 제거 완료
D:\Ramp Data\10,30 폴더의 제거 완료
D:\Ramp Data\20,10 폴더의 제거 완료
D:\Ramp Data\20,20 폴더의 제거 완료
D:\Ramp Data\20,30 폴더의 제거 완료
D:\Ramp Data\30,10 폴더의 제거 완료
D:\Ramp Data\30,20 폴더의 제거 완료
D:\Ramp Data\30,30 폴더의 제거 완료
모든 작업 완료


## 특정 폴더 아래의 모든 폴더, 파일 다 지우는 코드

In [10]:
import os

def delete_files_in_folders(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            os.remove(os.path.join(root, file))

# 사용 예시
folder_path = r'D:\0427Results(21,22,23,24,31,32,33,34,41,42,43,44)'
delete_files_in_folders(folder_path)

print('모든 파일, 폴더를 삭제했습니다. ')

모든 파일, 폴더를 삭제했습니다. 


## 얘는 걍 폴더 자체를 날려버리는구먼... 폴더 싹다 없애버림.. Scenario 폴더 하위에..

In [9]:
import os

def delete_folders_and_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            os.remove(os.path.join(root, file))
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            delete_folders_and_files(dir_path)
            os.rmdir(dir_path)

# 사용 예시
folder_path = r'C:\Users\user\Desktop\23.04.15\Scenarios'
delete_folders_and_files(folder_path)

print('모든 파일, 폴더를 삭제했습니다. ')

모든 파일, 폴더를 삭제했습니다. 


## 특정 폴더 내 CSV 파일을 Xlsx 파일로 바꿔주는 코드

In [59]:
import os
import pandas as pd

# CSV 파일을 XLSX로 변환하는 함수
def convert_csv_to_xlsx(csv_file_path, xlsx_file_path):
    data = pd.read_csv(csv_file_path)
    data.to_excel(xlsx_file_path, index=False)

    # 변환 완료 후 CSV 파일 삭제
    os.remove(csv_file_path)

# 변환할 폴더 경로
folder_path = rf'D:\0522Results(20,30,40 with CC2,CC4)\result\fitting'

# 폴더 내의 모든 파일에 대해 변환 및 삭제 수행
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        csv_file_path = os.path.join(folder_path, filename)
        xlsx_file_path = os.path.join(folder_path, filename[:-4] + '.xlsx')
        convert_csv_to_xlsx(csv_file_path, xlsx_file_path)
        print(f'{filename}을(를) 변환하여 {xlsx_file_path}로 저장했습니다.')
        print(f'{csv_file_path}을(를) 삭제했습니다.')


20_0_0-0_AVG.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_0_0-0_AVG.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_0_0-0_AVG.csv을(를) 삭제했습니다.
20_0_0-0_HGV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_0_0-0_HGV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_0_0-0_HGV.csv을(를) 삭제했습니다.
20_0_0-0_LOV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_0_0-0_LOV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_0_0-0_LOV.csv을(를) 삭제했습니다.
20_100_0-0_AVG.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_100_0-0_AVG.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_100_0-0_AVG.csv을(를) 삭제했습니다.
20_100_0-0_HGV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_100_0-0_HGV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\20_100_0-0_HGV.csv을(를) 삭제했습니다.
20_100_0-0_LOV.csv을(를) 변환하여 D:\0522Results(20,30

30_40_0-0_HGV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_40_0-0_HGV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_40_0-0_HGV.csv을(를) 삭제했습니다.
30_40_0-0_LOV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_40_0-0_LOV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_40_0-0_LOV.csv을(를) 삭제했습니다.
30_50_0-0_AVG.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_50_0-0_AVG.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_50_0-0_AVG.csv을(를) 삭제했습니다.
30_50_0-0_HGV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_50_0-0_HGV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_50_0-0_HGV.csv을(를) 삭제했습니다.
30_50_0-0_LOV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_50_0-0_LOV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\30_50_0-0_LOV.csv을(를) 삭제했습니다.
30_60_0-0_AVG.csv을(를) 변환하여 D:\0522Results(20,

40_80_0-0_LOV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_80_0-0_LOV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_80_0-0_LOV.csv을(를) 삭제했습니다.
40_90_0-0_AVG.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_90_0-0_AVG.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_90_0-0_AVG.csv을(를) 삭제했습니다.
40_90_0-0_HGV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_90_0-0_HGV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_90_0-0_HGV.csv을(를) 삭제했습니다.
40_90_0-0_LOV.csv을(를) 변환하여 D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_90_0-0_LOV.xlsx로 저장했습니다.
D:\0522Results(20,30,40 with CC2,CC4)\result\fitting\40_90_0-0_LOV.csv을(를) 삭제했습니다.


## 특정 폴더 내 rsr 파일 중 이름이 끝자리가 06-10으로 끝나는 애들을 01-05로 바꿔주는 코드

In [25]:
import os

def rename_rsr_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".rsr"):
                original_name = os.path.join(root, file)
                file_name, file_ext = os.path.splitext(file)
                last_two_chars = file_name[-2:]
                new_last_two_chars = ""

                if last_two_chars == "06":
                    new_last_two_chars = "01"
                elif last_two_chars == "07":
                    new_last_two_chars = "02"
                elif last_two_chars == "08":
                    new_last_two_chars = "03"
                elif last_two_chars == "09":
                    new_last_two_chars = "04"
                elif last_two_chars == "10":
                    new_last_two_chars = "05"
                else:
                    continue

                new_file_name = file_name[:-2] + new_last_two_chars
                new_name = os.path.join(root, new_file_name + file_ext)
                
                os.rename(original_name, new_name)

# 폴더 경로 설정
folder_path = rf"D:\0522Results(20,30,40 with CC2,CC4)_2nd\00,00"

# 함수 호출
rename_rsr_files(folder_path)


print('완료')

### 1,2,3,4,5만 있는지 check

In [25]:
import os

def rename_rsr_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".rsr"):
                original_name = os.path.join(root, file)
                file_name, file_ext = os.path.splitext(file)
                last_two_chars = file_name[-2:]
                new_last_two_chars = ""

                if not(last_two_chars == "01" or last_two_chars == "02" or last_two_chars == "03" or last_two_chars == "04" or last_two_chars == "05"):
                    print(file_name, '이상함')
                

# 폴더 경로 설정
folder_path = rf"D:\0522Results(20,30,40 with CC2,CC4)_2nd\00,00"

# 함수 호출
rename_rsr_files(folder_path)


print('이상한거 없음')

## .results 폴더 다 지워주는 코드

In [41]:
import os
import shutil

def delete_results_folders(folder_path):
    for root, dirs, files in os.walk(folder_path, topdown=False):
        for folder_name in dirs:
            folder = os.path.join(root, folder_name)
            if folder_name.endswith('.results'):
                shutil.rmtree(folder)

folder_path = r'C:\Users\user\Desktop\23.04.25(at 20, 30, 40 and truck ratio 0~100) - 복사본\Scenarios'
delete_results_folders(folder_path)


## AVG.csv 파일 다 지워주는 코드 (하위 폴더 전부)

In [20]:
import os

def delete_files_with_suffix(folder_path, suffix):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(suffix):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
    print('모두 지우기 완료')

# 주어진 폴더 경로와 파일 이름 끝에 붙는 접미사를 지정합니다.
folder_path = rf'D:\Straight Data\2040\result'  # 본인의 폴더 경로로 바꿔주세요.
suffix = '_AVG.csv'

delete_files_with_suffix(folder_path, suffix)

Deleted file: D:\Straight Data\2040\result\20_0_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_100_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_10_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_20_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_30_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_40_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_50_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_60_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_70_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_80_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\20_90_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\30_0_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\30_100_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\30_10_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\30_20_0-0_AVG.csv
Deleted file: D:\Straight Data\2040\result\30_30_0-0_AVG.csv
Deleted file: D:\Straigh

## 파일 옮기기 코드

In [158]:
import os
import shutil



def create_folders_and_copy_files(source_folder, target_folder):
    # 폴더 생성
    folders = ['2124', '3134', '4144']
    subfolders = ['LOV', 'HGV']

    for folder in folders:
        folder_path = os.path.join(target_folder, folder)
        if not os.path.exists(folder_path):  # 폴더가 이미 존재하는지 확인
            os.makedirs(folder_path)
            for subfolder in subfolders:
                subfolder_path = os.path.join(folder_path, subfolder)
                os.makedirs(subfolder_path)

    # 파일 복사
    files = os.listdir(source_folder)

    for file in files:
        target_path = None  # target_path 변수 초기화
        if file.startswith(('21', '22', '23', '24')):
            if file.endswith('_LOV.csv'):
                target_path = os.path.join(target_folder, '2124', 'LOV', file)
            elif file.endswith('_HGV.csv'):
                target_path = os.path.join(target_folder, '2124', 'HGV', file)

        elif file.startswith(('31', '32', '33', '34')):
            if file.endswith('_LOV.csv'):
                target_path = os.path.join(target_folder, '3134', 'LOV', file)
            elif file.endswith('_HGV.csv'):
                target_path = os.path.join(target_folder, '3134', 'HGV', file)

        elif file.startswith(('41', '42', '43', '44')):
            if file.endswith('_LOV.csv'):
                target_path = os.path.join(target_folder, '4144', 'LOV', file)
            elif file.endswith('_HGV.csv'):
                target_path = os.path.join(target_folder, '4144', 'HGV', file)

        if target_path:
            shutil.copy2(os.path.join(source_folder, file), target_path)

            
############################################################################################
# 입력받은 경로를 지정합니다.
base_folder = r'D:\Ramp Data'

# 폴더 생성을 위한 구간별 값들을 지정합니다.
ranges = ['10', '20', '30']
combinations = [f'{x},{y}' for x in ranges for y in ranges]


# 모든 폴더에 대해 반복
for combination in combinations:
    folder_path = os.path.join(base_folder, combination)
    source_folder = os.path.join(folder_path, 'result')
    target_folder = source_folder
            

    # 폴더 및 파일 복사 함수 호출
    create_folders_and_copy_files(source_folder, target_folder)
    print(combination, '완료')

print('완료')

10,10 완료
10,20 완료
10,30 완료
20,10 완료
20,20 완료
20,30 완료
30,10 완료
30,20 완료
30,30 완료
완료


## 이상하게 10-30 애들이 20,10 / 20,20/ 20,30/ 30,10/30,20/30,30에 들어가있다

In [82]:
import os

path = r"D:\Ramp Data\30,30\result\ALL"

# 경로 내 파일들을 검색하여 조건에 맞는 파일 삭제
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        cc_part = filename[-13:-8]  # 파일 이름에서 "C-C" 부분을 뒤에서부터 인덱싱
        if cc_part == "10-30":
            file_path = os.path.join(path, filename)
            print(file_path)
            os.remove(file_path)



D:\Ramp Data\30,30\result\ALL\21_0_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_0_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_100_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_100_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_10_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_10_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_20_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_20_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_30_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_30_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_40_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_40_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_50_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_50_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_60_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_60_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_70_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_70_10-30_LOV.csv
D:\Ramp Data\30,30\result\ALL\21_80_10-30_HGV.csv
D:\Ramp Data\30,30\result\ALL\21_80_10-30_LOV.csv


In [97]:
import os

# 파일 경로 설정
folder_path = r'D:\Ramp Data\10,10\result\2124\LOV - 복사본'

# 폴더 내의 모든 파일 확인
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv'):
            # 파일 이름 분리
            filename, extension = os.path.splitext(file)
            
            # 파일 이름을 A, B, C, D로 분할
            A, B, C_D, XXX = filename.split('_')
            
            # A를 10으로 나눈 나머지에 따라 D 또는 C를 0으로 변경
            remainder = int(A) % 10
            if remainder == 1:
                print(C_D)
                filename[-10:-8] = '0'
            elif remainder == 2:
                filename[-13:-11] = '0'
            else:
                continue
            
            
            # 변경된 파일 이름 출력
            print(filename)


10-10


TypeError: 'str' object does not support item assignment

In [99]:
import os

# 파일 경로 설정
folder_path = r'D:\Ramp Data\10,10\result\2124\LOV - 복사본'

# 폴더 내의 모든 파일 확인
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv'):
            # 파일 이름 분리
            filename, extension = os.path.splitext(file)
            
            # 파일 이름을 A, B, C, D로 분할
            A, B, C_D, XXX = filename.split('_')
            
            # A를 10으로 나눈 나머지에 따라 D 또는 C를 0으로 변경
            remainder = int(A) % 10
            if remainder == 1:
                C_D = C_D.replace('D', '0')
            elif remainder == 2:
                C_D = C_D.replace('C', '0')
            
            # 변경된 파일 이름 재구성
            new_filename = '_'.join([A, B, C_D, XXX]) + extension
            print(new_filename)
            
            # 변경된 파일 경로 생성
            new_file_path = os.path.join(root, new_filename)
            file_path = os.path.join(root, file)
            
            # 변경된 파일 이름으로 파일 이동
            os.rename(file_path, new_file_path)


21_0_10-10_LOV.csv
21_100_10-10_LOV.csv
21_10_10-10_LOV.csv
21_20_10-10_LOV.csv
21_30_10-10_LOV.csv
21_40_10-10_LOV.csv
21_50_10-10_LOV.csv
21_60_10-10_LOV.csv
21_70_10-10_LOV.csv
21_80_10-10_LOV.csv
21_90_10-10_LOV.csv
22_0_10-10_LOV.csv
22_100_10-10_LOV.csv
22_10_10-10_LOV.csv
22_20_10-10_LOV.csv
22_30_10-10_LOV.csv
22_40_10-10_LOV.csv
22_50_10-10_LOV.csv
22_60_10-10_LOV.csv
22_70_10-10_LOV.csv
22_80_10-10_LOV.csv
22_90_10-10_LOV.csv
23_0_10-10_LOV.csv
23_100_10-10_LOV.csv
23_10_10-10_LOV.csv
23_20_10-10_LOV.csv
23_30_10-10_LOV.csv
23_40_10-10_LOV.csv
23_50_10-10_LOV.csv
23_60_10-10_LOV.csv
23_70_10-10_LOV.csv
23_80_10-10_LOV.csv
23_90_10-10_LOV.csv
24_0_10-10_LOV.csv
24_100_10-10_LOV.csv
24_10_10-10_LOV.csv
24_20_10-10_LOV.csv
24_30_10-10_LOV.csv
24_40_10-10_LOV.csv
24_50_10-10_LOV.csv
24_60_10-10_LOV.csv
24_70_10-10_LOV.csv
24_80_10-10_LOV.csv
24_90_10-10_LOV.csv
